In [2]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
from scipy.stats import mannwhitneyu,ttest_ind, kruskal,chisquare, pearsonr, spearmanr

In [3]:
def Basic_statistics_pipeline(mean1, mean2, sd1, sd2):
    """набор классических статистических тестов в библиотеке сайпай
        
        
        В медицине, обычно, мы анализируем тип распределения: нормальное или не нормальное, а
        а затем выбираем тесты, которые могут использоваться в данном случае. 
        
        Здесь, я просто сгенерировал нормальное распределение и провёл тесты. Позже мы научимся
        как доставать из данных значения и проводить тесты на них
    """
  
    lst_1 = np.random.normal(mean1, sd1, 1000) # генерация нормального распределения
    lst_2 = np.random.normal(mean2, sd2, 1000) # генерация нормального распределения
    res_tt = stats.ttest_ind(lst_1, lst_2) # т-тест стьюдента
    res_mw = mannwhitneyu(lst_1, lst_2, 
                       use_continuity=True, 
                       alternative=None) # u -тест манна-уитни
    res_kw = kruskal(lst_1, lst_2)## тест краскела-уолиса
    res_chi = chisquare(lst_1, lst_2) ## хи-квадрат
    res_pearson = pearsonr(lst_1, lst_2)# корреляция пирсона
    res_spear = spearmanr(lst_1, lst_2)# корреляция спирмана
    res1_ks = stats.kstest(lst_1,'norm',alternative='greater')#тест колмагорова-смирнова ( показывает 
                                                            #нормальность или не нормальность распределения)
    res2_ks = stats.kstest(lst_2,'norm',alternative='greater')#тест колмагорова-смирнова
    shapiro_test = stats.shapiro(lst_2)#тест шапиро-уилка( показывает 
                                                     #нормальность или не нормальность распределения)
    anova = stats.f_oneway(lst_1, lst_2)
    oddsratio, pvalue = stats.fisher_exact([[11200, 100], [80, 5]]) # точный тест фишера
    return anova, oddsratio, pvalue, res_tt, res_mw, res_kw, res_chi, res_pearson, res_spear, res1_ks, res2_ks, shapiro_test
    
    
    
Basic_statistics_pipeline(150, 46, 5, 3)

(F_onewayResult(statistic=278936.27519928257, pvalue=0.0),
 7.0,
 0.0011084393833355923,
 Ttest_indResult(statistic=528.1441803137504, pvalue=0.0),
 MannwhitneyuResult(statistic=0.0, pvalue=0.0),
 KruskalResult(statistic=1499.2503748125928, pvalue=0.0),
 Power_divergenceResult(statistic=235374.97459822835, pvalue=0.0),
 (0.006593068106711857, 0.8350473404452077),
 SpearmanrResult(correlation=-0.00021686421686421687, pvalue=0.9945351147314208),
 KstestResult(statistic=0.0, pvalue=1.0),
 KstestResult(statistic=0.0, pvalue=1.0),
 (0.9993940591812134, 0.9913946390151978))

In [4]:
# импорт файла в пандас
# можно читать много разных форматов. csv = данные, разделённые запятыми наиболее часто используется
data = pd.read_csv('data_stroke.csv')

In [5]:
# посмотреть первые 5 или 7 или 10 строк
data.head(5)

,Gender,Age,AF,Localisation,AMI_Unst.ang,GFR,Surv,Q_non_Q,DM
0,male,56,No_AF,Left_car,AMI,14,death,non_Q,Diabetes
1,male,59,No_AF,Left_car,Unst_ang,98,alive,NaN,No_Diabetes
2,female,86,AF,Left_car,AMI,57,death,non_Q,Diabetes
3,female,83,AF,Left_car,Unst_ang,69,alive,NaN,No_Diabetes
4,female,78,AF,Left_car,AMI,54,death,non_Q,No_Diabetes


In [6]:
#выбор столбца - просто пишем дата и в квадратных скобках указываем название столбца
data['Age'] # data.Age => также

0      56
1      59
2      86
3      83
4      78
       ..
108    78
109    73
110    68
111    81
112    84
Name: Age, Length: 113, dtype: int64

In [7]:
# например, я хочу узнать сколько в моих данных мужчин и женщин, для этого можно использовать метод
data['Gender'].value_counts()

female    83
male      30
Name: Gender, dtype: int64

In [9]:
# например, уникальные значения столбца
data['Localisation'].unique()

array(['Left_car', 'Right_car', 'Vertebro_bas'], dtype=object)

In [10]:
# узнать названия всех столбцов
data.columns

Index(['Gender', 'Age', 'AF', 'Localisation', 'AMI_Unst.ang', 'GFR', 'Surv',
       'Q_non_Q', 'DM'],
      dtype='object')

In [11]:
# выборка данных 1) используя метод loc ( локализация)
data.loc[1:5, 'AF': 'DM'] # первые значения 1 и 5 - это строки, которыя я хотел, а АФ и ДМ это столбцы, которые меня
# интересуют

,AF,Localisation,AMI_Unst.ang,GFR,Surv,Q_non_Q,DM
1,No_AF,Left_car,Unst_ang,98,alive,NaN,No_Diabetes
2,AF,Left_car,AMI,57,death,non_Q,Diabetes
3,AF,Left_car,Unst_ang,69,alive,NaN,No_Diabetes
4,AF,Left_car,AMI,54,death,non_Q,No_Diabetes
5,No_AF,Right_car,AMI,86,alive,NaN,No_Diabetes


In [12]:
# выборка данных 2) используя метод iloc (числовая локализация)
data.iloc[1:3, 2:5] # выбок 1-3 строк (не включая) и 2-5 столбцов (5 не включая)

,AF,Localisation,AMI_Unst.ang
1,No_AF,Left_car,Unst_ang
2,AF,Left_car,AMI


In [13]:
# мы хотим сделать "подданные", например состоящие только из пола и диабета
df_new = data[['Gender', 'DM']]
df_new.head(3)

,Gender,DM
0,male,Diabetes
1,male,No_Diabetes
2,female,Diabetes


In [14]:
# мы хотим сделать выбрать данные только для умерших пациентов
GFR_death = data[data['Surv'] == 'death']
GFR_death.head(5)

,Gender,Age,AF,Localisation,AMI_Unst.ang,GFR,Surv,Q_non_Q,DM
0,male,56,No_AF,Left_car,AMI,14,death,non_Q,Diabetes
2,female,86,AF,Left_car,AMI,57,death,non_Q,Diabetes
4,female,78,AF,Left_car,AMI,54,death,non_Q,No_Diabetes
6,female,76,AF,Right_car,AMI,12,death,NaN,Diabetes
8,male,78,AF,Left_car,AMI,32,death,NaN,Diabetes


In [15]:
# описание данных - возвращает базовые статистики из численных данных
data.describe()

,Age,GFR
count,113.000000,113.000000
mean,77.017699,43.000000
std,8.869070,20.341285
min,56.000000,9.000000
25%,71.000000,29.000000
50%,78.000000,40.000000
75%,83.000000,54.000000
max,98.000000,98.000000


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Gender        113 non-null    object
 1   Age           113 non-null    int64 
 2   AF            113 non-null    object
 3   Localisation  113 non-null    object
 4   AMI_Unst.ang  113 non-null    object
 5   GFR           113 non-null    int64 
 6   Surv          113 non-null    object
 7   Q_non_Q       54 non-null     object
 8   DM            113 non-null    object
dtypes: int64(2), object(7)
memory usage: 8.1+ KB


In [17]:
# группировка этих данных по столбцу/переменной Surv
# возвращает объект DataFrameGroupBy
grouped_by_surv = data.groupby('Surv')
print(grouped_by_surv)

In [18]:
grouped_by_surv.ngroups

2

In [19]:
grouped_by_surv.groups

{'alive': [1, 3, 5, 7, 10, 14, 15, 20, 23, 25, 26, 27, 30, 32, 34, 35, 40, 41, 42, 43, 46, 47, 53, 54, 56, 59, 60, 66, 69, 71, 72, 73, 76, 77, 79, 80, 81, 84, 85, 91, 92, 94, 95, 97, 98, 99, 101, 102, 106, 110, 111, 112], 'death': [0, 2, 4, 6, 8, 9, 11, 12, 13, 16, 17, 18, 19, 21, 22, 24, 28, 29, 31, 33, 36, 37, 38, 39, 44, 45, 48, 49, 50, 51, 52, 55, 57, 58, 61, 62, 63, 64, 65, 67, 68, 70, 74, 75, 78, 82, 83, 86, 87, 88, 89, 90, 93, 96, 100, 103, 104, 105, 107, 108, 109]}

In [20]:
# получаем данные конкретной группы
grouped_by_surv.get_group('death')[:5]

,Gender,Age,AF,Localisation,AMI_Unst.ang,GFR,Surv,Q_non_Q,DM
0,male,56,No_AF,Left_car,AMI,14,death,non_Q,Diabetes
2,female,86,AF,Left_car,AMI,57,death,non_Q,Diabetes
4,female,78,AF,Left_car,AMI,54,death,non_Q,No_Diabetes
6,female,76,AF,Right_car,AMI,12,death,NaN,Diabetes
8,male,78,AF,Left_car,AMI,32,death,NaN,Diabetes


In [21]:
# давайте проведём простой тест. Пока не будет вдаваться в подробности распределения, давайте проверим разными
# тестами имеет ли место статистически значимая разница возраста между умершими и выжившими 
data_survive = data[data.Surv == 'alive']
data_dead = data[data.Surv == 'death']

In [22]:
# res_kw = kruskal(lst_1, lst_2)## тест краскела-уолиса

res_kw = kruskal(data_survive['GFR'], data_dead['GFR'])
print(res_kw)

KruskalResult(statistic=19.1875063598709, pvalue=1.1848633731444293e-05)


In [23]:
# р-значение 1.18, т.е. нулевую гипотезу, о том, что между данными нет разницы - мы не отвергаем...

In [ ]:
"""
1) Задание - сделать подвыборку в зависимости от наличия или отсутствия фибрилляции предсердий (AF)
2) Задание - проверить стат.гипотезы: нет разницы между уровенм СКФ (GFR) у умерших и выживших
3) Задание - проверить тип расспределения при помощи методов Колмогорова-Смирнова и Шапиро-Уилка (Возраст, СКФ)
4) Подумать где можно реализовать Хи-квадрат и проверить произвольную гипотезу и сделать вывод о проделанной работе

"""

In [ ]:
af_af = data[data['AF'] == 'AF']
af_af.head(10)

In [ ]:
no_af = data[data['AF'] == 'No_AF']
no_af.head(10)

In [26]:
res_kw_age = kruskal(data_survive['Age'], data_dead['Age'])
res_kw_age

KruskalResult(statistic=2.3886525182904954, pvalue=0.12221893625904245)

In [ ]:
# р = 0,122 говорит, что разница есть. Отвергаем нулевую гипотезу

In [27]:
res_tt = stats.ttest_ind(data_survive['Age'], data_dead['Age'])
res_tt

Ttest_indResult(statistic=-1.5398869481546216, pvalue=0.1264338826722581)

In [ ]:
from scipy.stats import kstest
kstest(data['Age'], 'norm')

KstestResult(statistic=1.0, pvalue=0.0)

In [ ]:
kstest(data['GFR'], 'norm')

KstestResult(statistic=1.0, pvalue=0.0)

In [ ]:
res_shapiro_age = stats.shapiro(data['Age'])
res_shapiro_age
# p больше альфа уровня. 

(0.9839508533477783, 0.19562266767024994)

In [ ]:
res_shapiro_gfr = stats.shapiro(data['GFR'])
res_shapiro_gfr

(0.9576783180236816, 0.0012525040656328201)

In [ ]:
# если р меньше заданного альфа уровня (0.1, 0.05) нулевая гипотеза о нормальном распределении отвергается. Данные имеют значимость.

In [ ]:
res_chi = chisquare(lst_1, lst_2)

Power_divergenceResult(statistic=8191.670496731499, pvalue=0.0)

In [ ]:
# ищем зависимость показателя СКФ от возраста. отвергая нулевую гипотезу, мы ошибемся на 0%. значит, зависимость есть.

In [40]:
data['Gender'].value_counts()

female    83
male      30
Name: Gender, dtype: int64

In [35]:
gender_fem = data[data['Gender'] == 'female']
gender_fem['DM'].value_counts()

No_Diabetes    52
Diabetes       31
Name: DM, dtype: int64

In [36]:
gender_m = data[data['Gender'] == 'male']
gender_m['DM'].value_counts()

No_Diabetes    20
Diabetes       10
Name: DM, dtype: int64

In [41]:
lst_1 = [83, 30]
lst_2 = [31, 10]
res_chi = chisquare(lst_1, lst_2)

In [42]:
res_chi

Power_divergenceResult(statistic=127.2258064516129, pvalue=1.657947013449988e-29)